# **HDF5 to Delta Lake Data Conversion**
The raw data files provided by Bosch are in HDF5 (.h5) format, a hierarchical data format commonly used for storing large numerical datasets, especially in scientific and industrial applications. While powerful for structured, multi-dimensional data, HDF5 is increasingly considered outdated for use in modern distributed data systems — particularly due to its limited compatibility with big data tools and lack of built-in support for scalable query engines like Spark.

The procedures used here can be easily adapted for other semi-structured formats such as CSV or JSON. The main logic of reading, transforming, and storing structured data remains the same.

Our goal in this notebook is to:
- Convert each .h5 file into a row-oriented format suitable for distributed processing — in this case, Parquet files.
- Store these files as a Delta Lake table, which provides fast, reliable access to large-scale data using Apache Spark.
- Implement a partitioning strategy that enables efficient filtering and fast retrieval, based on the structure and metadata of the original files.

This notebook represents Phase 1 of the pipeline: ingestion and conversion from raw industrial data into a format optimized for analytics and machine learning.

⚠️ Note: Before running this notebook, make sure you have completed **Step 0.2** of the **Phase 0**. This involves downloading the raw data from the `CNC_Machining` repository and placing it in the data/raw/ directory. The code in this notebook relies on the presence of these .h5 files in that location.

In [1]:
import os
import time
import h5py
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, StringType
from pyspark.sql.functions import col
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from tqdm import tqdm
import gc

In [2]:
# Directory where raw data is stored
RAW_DIR = os.path.join("..", "data", "raw")
# Directory where Delta data is stored
DELTA_DIR = os.path.join("..", "data", "delta")

In [3]:
# File for logging any failed conversions from hdf5 to Delta
FAILED_LOG = os.path.join("..", "data", "failed_files.txt")
SUCCESS_LOG = os.path.join("..", "data", "success_files.txt")

# Clean failed and success logs if they exists
open(FAILED_LOG, "w").close()
open(SUCCESS_LOG, "w").close()

In [4]:
# Number of workers for writing files to Delta
MAX_WORKERS = 4  # You can increase this based on system resources

In [5]:
# Used to store number of files per thread
thread_data = threading.local()

In [6]:
# Check for hd5 files - fail if none found
h5_files = [
    os.path.join(root, f)
    for root, _, files in os.walk(RAW_DIR)
    for f in files if f.endswith(".h5")
]

assert len(h5_files) > 0, "⚠️ No .h5 files found in the raw data directory. Did you complete step 0.2?"
print(f"✅ Found {len(h5_files)} .h5 files in the raw data directory.")

✅ Found 1702 .h5 files in the raw data directory.


## 📊 **Exploring Raw File Structure to Guide Delta Lake Partitioning**
In this section, we collect metadata from each HDF5 file — including dataset keys, shapes, and types — to understand the structure and volume of data across files. This exploration helps us make an informed decision about how to partition the Delta Lake table. Partitioning is a way of organizing data by one or more key columns (e.g., machine ID, operation, or date) to improve query performance by enabling Spark to read only the relevant subsets of data. However, deep or overly granular partitioning (such as by exact timestamp or unique file ID) can backfire, creating many tiny files and directories that slow down query planning and metadata operations. Our goal is to find the right balance: enough partitioning to enable fast filtering, but not so much that it fragments the dataset.

In [7]:
# Function to extract metadata from HDF5 files and save to DataFrame
def collect_hdf5_metadata(base_dir):
    data = []

    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".h5"):
                file_path = os.path.join(root, file)
                record = {
                    "file_path": file_path,
                    "num_rows": None,
                    "num_columns": None,
                    "column_names": []
                }

                try:
                    with h5py.File(file_path, 'r') as f:
                        for key in f.keys():
                            dataset = f[key]
                            if len(dataset.shape) == 2:  # Ensure it's a 2D dataset
                                record["num_rows"] = dataset.shape[0]
                                record["num_columns"] = dataset.shape[1]
                                record["column_names"] = list(dataset.attrs.get("column_names", []))
                                break  # Process only the first 2D dataset
                except Exception as e:
                    record["error"] = str(e)

                data.append(record)

    return pd.DataFrame(data)

In [8]:
# Run and save result
df_files = collect_hdf5_metadata(RAW_DIR)

In [9]:
# View summary
df_files.head()

,file_path,num_rows,num_columns,column_names
0,../data/raw/M01/OP03/bad/M01_Aug_2019_OP03_000.h5,139653,3,[]
1,../data/raw/M01/OP03/good/M01_Feb_2020_OP03_00...,179200,3,[]
2,../data/raw/M01/OP03/good/M01_Aug_2019_OP03_00...,178176,3,[]
3,../data/raw/M01/OP03/good/M01_Feb_2019_OP03_00...,156000,3,[]
4,../data/raw/M01/OP03/good/M01_Feb_2020_OP03_00...,178176,3,[]


In [10]:
print("Mean number of rows and columns:")
print(df_files[['num_rows', 'num_columns']].mean())
print("Number of files:")
print(df_files['file_path'].nunique())


Mean number of rows and columns:
num_rows       104936.098707
num_columns         3.000000
dtype: float64
Number of files:
1702


## 📌 **Data Structure and Partitioning Strategy**
Each HDF5 file contains time-series sensor data with three columns: "x", "y", and "z", representing vibration signals along three axes. This structure is described in the original `CNC_Machining` [GitHub repository](https://github.com/boschresearch/CNC_Machining/blob/main/README.md).

To efficiently store and query this data using Delta Lake, we will apply partitioning — a strategy that organizes the dataset into subfolders based on selected column values. Partitioning allows Spark to read only the relevant slices of data when filtering, significantly improving performance on large datasets.

Based on the file structure and metadata, we will partition the Delta table by:

- `machine_id` (e.g., M01, M02)
- `operation` (e.g., OP00, OP01)
- `label` (e.g., good, bad)

This strikes a balance between fast filtering and avoiding excessive small files or deeply nested directories.

In [11]:
# Init Spark
spark = SparkSession.builder \
    .appName("Delta Lake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [12]:
# Define a schema for the spark dataframe to ensure consistency
# across all files
schema = StructType([
    StructField("x", DoubleType(), True),
    StructField("y", DoubleType(), True),
    StructField("z", DoubleType(), True),
    StructField("machine_id", StringType(), True),
    StructField("month", StringType(), True),
    StructField("year", StringType(), True),
    StructField("operation", StringType(), True),
    StructField("example_no", StringType(), True),
    StructField("label", StringType(), True),
])


In [13]:
# Function to parse information from the file names
def parse_filename(filename):
    base = filename.replace(".h5", "")
    parts = base.split("_")
    return {
        "machine_id": parts[0],
        "month": parts[1],
        "year": parts[2],
        "operation": parts[3],
        "example_no": parts[4]
    }

In [14]:
# Function to process each file
def process_file(file_path):
    start = time.time()
    try:
        # Initialize per-thread counter if not already set
        if not hasattr(thread_data, "file_count"):
            thread_data.file_count = 0

        # Parse metadata from file name and path
        meta = parse_filename(os.path.basename(file_path))
        label = os.path.basename(os.path.dirname(file_path))

        # Check if already written to Delta
        delta_df = spark.read.format("delta").load(DELTA_DIR)
        exists = delta_df.filter(
            (col("machine_id") == meta["machine_id"]) &
            (col("operation") == meta["operation"]) &
            (col("example_no") == meta["example_no"]) &
            (col("year") == meta["year"]) &
            (col("month") == meta["month"]) &
            (col("label") == label)
        ).limit(1).count() > 0

        if exists:
            duration = round(time.time() - start, 2)
            return_msg = f"⏭️ Skipped (already exists): {file_path} ({duration}s)"
        else:
            with h5py.File(file_path, 'r') as h5f:
                if "vibration_data" not in h5f:
                    raise ValueError("Missing 'vibration_data'")
                data = h5f["vibration_data"][()]
                df_pd = pd.DataFrame(data, columns=["x", "y", "z"])
                df_pd[["x", "y", "z"]] = df_pd[["x", "y", "z"]].astype("float64")

                for k, v in meta.items():
                    df_pd[k] = v
                df_pd["label"] = label

                df_spark = spark.createDataFrame(df_pd, schema=schema)

                df_spark.write.format("delta") \
                    .partitionBy("machine_id", "operation", "label") \
                    .mode("append") \
                    .save(DELTA_DIR)

            # Manual cleanup only every 25 files per thread
            thread_data.file_count += 1
            if thread_data.file_count % 25 == 0:
                del df_pd
                del df_spark
                spark.catalog.clearCache()
                gc.collect()

            duration = round(time.time() - start, 2)
            return_msg = f"✅ {file_path} ({duration}s)"

        with open(SUCCESS_LOG, "a") as log:
            log.write(f"{return_msg}\n")

        return return_msg

    except Exception as e:
        duration = round(time.time() - start, 2)
        error_msg = f"⚠️ {file_path} FAILED after {duration}s: {e}"
        with open(FAILED_LOG, "a") as log:
            log.write(f"{file_path} | {e}\n")
        return error_msg

In [15]:
# Build list of all .h5 files
h5_files = []
for root, _, files in os.walk(RAW_DIR):
    for f in files:
        if f.endswith(".h5"):
            h5_files.append(os.path.join(root, f))

In [16]:
# To speed this up, we can use threading
# Run threads with progress bar
# Number of workers defined at top of file
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(process_file, f) for f in h5_files]
    for result in tqdm(as_completed(futures), total=len(h5_files), desc="Processing files"):
        # Output was too much, so commented out this line
        #print(result.result())
        pass

Processing files: 100%|██████████| 1702/1702 [1:05:23<00:00,  2.31s/it]


In [17]:
# Stop the Spark session
spark.stop()

**See the next notebook- `verify_delta_lake.ipynb`, to verify the data in the Delta Lake table is as expected.**